<a href="https://colab.research.google.com/github/CS222-UIUC/course-project-movie-recommender/blob/main/Hkang_Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import sklearn as sk
import numpy as np
import pandas as pd
import tensorflow as ts
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
df = pd.read_csv("movies.csv")
df.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young gi...,3422.537,Blinding Edge Pictures-Universal Pictures-Film...,2023-02-01,20000000.0,52000000.0,100.0,Released,Save your family or save humanity. Make the ch...,6.457,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kris...,based on novel or book-sacrifice-cabin-faith-e...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-104...
1,646389,Plane,Action-Adventure-Thriller,en,After a heroic job of successfully landing his...,2618.646,MadRiver Pictures-Di Bonaventura Pictures-G-BA...,2023-01-12,25000000.0,51000000.0,107.0,Released,Survive together or die alone.,6.901,785.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,pilot-airplane-philippines-held hostage-plane ...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg,/9Rq14Eyrf7Tu1xk0Pl7VcNbNh1n.jpg,505642-758769-864692-631842-1058949-925943-758...
2,315162,Puss in Boots: The Last Wish,Animation-Adventure-Comedy-Family,en,Puss in Boots discovers that his passion for a...,2589.341,DreamWorks Animation-Universal Pictures,2022-12-07,90000000.0,442000000.0,103.0,Released,Say hola to his little friends.,8.402,4357.0,Antonio Banderas-Salma Hayek-Harvey Guillén-Wa...,fairy tale-talking dog-spin off-aftercreditsst...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg,/b1Y8SUb12gPHCSSSNlbX4nB3IKy.jpg,536554-76600-816904-653851-899112-985335-83078...
3,505642,Black Panther: Wakanda Forever,Action-Adventure-Science Fiction,en,Queen Ramonda Shuri M’Baku Okoye and the Dora ...,2525.408,Marvel Studios,2022-11-09,250000000.0,858535561.0,162.0,Released,Forever.,7.338,3922.0,Letitia Wright-Lupita Nyong'o-Danai Gurira-Win...,loss of loved one-hero-sequel-superhero-based ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg,/xDMIl84Qo5Tsu62c9DGWhmPI67A.jpg,436270-829280-76600-56969-312634-1037858-238-5...
4,956101,The Eighth Clause,Thriller,la,Kat and Borja appear to be a perfect couple bu...,2259.303,SDB Films-El Hombre Orquesta,2022-04-29,0.0,0.0,0.0,Released,NaN,4.600,10.0,Maite Perroni-Manuel Vega-Óscar Jaenada-Jessic...,NaN,/8tc8eMFAX2SDC1TRu987qFQy8Cl.jpg,/kLnqNE9Af5QHyvUxw8cDGhF1ilv.jpg,NaN


In [27]:
print(df.shape)

(723029, 20)


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723029 entries, 0 to 723028
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    723029 non-null  int64  
 1   title                 723025 non-null  object 
 2   genres                511997 non-null  object 
 3   original_language     723029 non-null  object 
 4   overview              604350 non-null  object 
 5   popularity            723029 non-null  float64
 6   production_companies  337186 non-null  object 
 7   release_date          670295 non-null  object 
 8   budget                723029 non-null  float64
 9   revenue               723029 non-null  float64
 10  runtime               688534 non-null  float64
 11  status                723029 non-null  object 
 12  tagline               108057 non-null  object 
 13  vote_average          723029 non-null  float64
 14  vote_count            723029 non-null  float64
 15  

In [29]:
df.isnull().sum() # check null values

id                           0
title                        4
genres                  211032
original_language            0
overview                118679
popularity                   0
production_companies    385843
release_date             52734
budget                       0
revenue                      0
runtime                  34495
status                       0
tagline                 614972
vote_average                 0
vote_count                   0
credits                 225194
keywords                512782
poster_path             185469
backdrop_path           500639
recommendations         688347
dtype: int64

In [30]:
df.duplicated().sum()

4

In [31]:
df = df.drop(["production_companies", "popularity", "budget", "revenue", "status", "recommendations", "runtime", "vote_average", "backdrop_path",'tagline'], axis = 1) #'poster_path'], axis=1)

In [32]:
df.drop_duplicates(inplace=True)

In [33]:
df.title.duplicated().sum()

86984

In [34]:
df[["title", "release_date"]].duplicated().sum()# checking for release data with same movie title

2271

In [35]:
df.drop_duplicates(subset=["title","release_date"], inplace=True)

In [36]:
df = df[df.vote_count >= 350].reset_index()

In [37]:
df.isnull().sum()

index                  0
id                     0
title                  0
genres                 0
original_language      0
overview               1
release_date           0
vote_count             0
credits                8
keywords             238
poster_path            0
dtype: int64

In [43]:
df.fillna("", inplace = True) # replace the null values with ''
# We do have an option filling all the nulls with 0, but since they will have string values, I thought that na -> "" will be necessary

In [44]:
index = df[(df.genres == "") & (df.overview == "")].index
df.drop(index, inplace=True)

In [45]:
df.genres = df.genres.apply(lambda x: " ".join(x.split("-")))
df.keywords = df.keywords.apply(lambda x: " ".join(x.split("-")))
df.credits = df.credits.apply(lambda x: " ".join(x.replace(" ", "").split("-")[:5]))

In [46]:
df["tags"] =df.overview + " "+ df.genres + " "  +df.credits + " " +df.keywords + " " + df.original_language

In [49]:
df_new = df[["id", "title", "tags", 'poster_path']]

In [51]:
df_new.tags = df_new.tags.apply(lambda x:x.lower())

In [52]:
df_new.tags[0]

'while vacationing at a remote cabin a young girl and her two fathers are taken hostage by four armed strangers who demand that the family make an unthinkable choice to avert the apocalypse. with limited access to the outside world the family must decide what they believe before all is lost. horror mystery thriller davebautistajonathangroffbenaldridgekristencuinikkiamuka based on novel or book sacrifice cabin faith end of the world apocalypse home invasion lgbt aftercreditsstinger adopted child adopted daughter shot on film gay same sex relationship religious symbolism en'

In [54]:
df_new.head()

,id,title,tags,poster_path
0,631842,Knock at the Cabin,while vacationing at a remote cabin a young gi...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg
1,646389,Plane,after a heroic job of successfully landing his...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg
2,315162,Puss in Boots: The Last Wish,puss in boots discovers that his passion for a...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg
3,505642,Black Panther: Wakanda Forever,queen ramonda shuri m’baku okoye and the dora ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg
4,937278,A Man Called Otto,when a lively young family moves in next door ...,/130H1gap9lFfiTF9iDrqNIkFvC9.jpg


In [56]:
prts = PorterStemmer()

In [57]:
def stem(text):
    y = []
    for i in text.split():
        y.append(prts.stem(i))
    
    return " ".join(y)    

In [58]:
df_new["tags"] = df_new["tags"].apply(stem)

<ipython-input-58-6e8e4450f3ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["tags"] = df_new["tags"].apply(stem)


In [60]:
countvec = CountVectorizer(stop_words="english",max_features=5000)

In [61]:
vect = countvec.fit_transform(df_new["tags"]).toarray()

In [64]:
countvec.get_feature_names_out()

array(['10', '100', '1000', ..., 'zombies', 'zone', 'zoo'], dtype=object)

In [66]:
sim = cosine_similarity(vect)

In [67]:
sim.shape

(7618, 7618)

In [ ]:
#now it is time for building recommendation system.